## 0. 源码
- framework/base/core/java/andorid/os/
  - Handler.java
  - Looper.java
  - Message.java
  - MessageQueue.java

---

## 1. 概述
- 在整个 Android 的源码世界里，有两大利剑，其一是 Binder IPC 机制，，另一个便是消息机制(由Handler/Looper/MessageQueue等构成的)
- Android 有大量的消息驱动方式来进行交互，比如 Android 的四剑客 Activity, Service, Broadcast, ContentProvider 的启动过程的交互

### 1.1 模型
- 消息机制主要包含：
  - Message：消息分为硬件产生的消息(如按钮、触摸)和软件生成的消息
  - MessageQueue：消息队列的主要功能向消息池投递消息(MessageQueue.enqueueMessage)和取走消息池的消息(MessageQueue.next)
  - Handler：消息辅助类，主要功能向消息池发送各种消息事件(Handler.sendMessage)和处理相应消息事件(Handler.handleMessage)
  - Looper：不断循环执行(Looper.loop)，按分发机制将消息分发给目标处理者
  
### 1.2 架构图
- Looper 有一个 MessageQueue 消息队列
- MessageQueue 有一组待处理的 Message
- Message 中有一个用于处理消息的 Handler
- Handler 中有 Looper 和 MessageQueue

![image](handler1_page1.png)

### 1.3 典型实例
```
class LooperThread extends Thread {
    public Handler mHandler;

    public void run() {
        Looper.prepare();   //【见 2.1】

        mHandler = new Handler() {  //【见 3.1】
            public void handleMessage(Message msg) {
                //TODO 定义消息处理逻辑. 【见 3.2】
            }
        };

        Looper.loop();  //【见 2.2】
    }
}
```

---

## 2. Looper
### 2.1 prepare()
- 对于无参的情况，默认调用 prepare(true)，表示的是这个 Looper 运行退出，而对于 false 的情况则表示当前 Looper 不运行退出
```
private static void prepare(boolean quitAllowed) {
    // 每个线程只允许执行一次该方法，第二次执行时线程的TLS已有数据，则会抛出异常。
    if (sThreadLocal.get() != null) {
        throw new RuntimeException("Only one Looper may be created per thread");
    }
    // 创建Looper对象，并保存到当前线程的TLS区域
    sThreadLocal.set(new Looper(quitAllowed));
}
```

- ThreadLocal：线程本地存储区（Thread Local Storage，简称为TLS），每个线程都有自己的私有的本地存储区域，不同线程之间彼此不能访问对方的 TLS 区域
```
static final ThreadLocal<Looper> sThreadLocal = new ThreadLocal<Looper>()
```

- Looper.prepare() 在每个线程只允许执行一次，该方法会创建 Looper 对象，Looper 的构造方法中会创建一个 MessageQueue 对象，再将 Looper 对象保存到当前线程 TLS
```
private Looper(boolean quitAllowed) {
    mQueue = new MessageQueue(quitAllowed);  // 创建 MessageQueue 对象. 【见4.1】
    mThread = Thread.currentThread();        // 记录当前线程.
}
```

- 还有一个 prepareMainLooper() 方法，该方法主要在 ActivityThread 类中使用
```
public static void prepareMainLooper() {
    prepare(false);    // 设置不允许退出的 Looper
    synchronized (Looper.class) {
        // 将当前的 Looper 保存为主 Looper，每个线程只允许执行一次。
        if (sMainLooper != null) {
            throw new IllegalStateException("The main Looper has already been prepared.");
        }
        sMainLooper = myLooper();
    }
}
```

### 2.2 loop()
- loop() 进入循环模式，不断重复下面的操作，直到没有消息时退出循环
  - 读取 MessageQueue 的下一条 Message
  - 把 Message 分发给相应的 target
  - 再把分发后的 Message 回收到消息池，以便重复利用

- 代码中可以看到有 logging 方法，这是用于 debug 的，默认情况下 logging == null，通过设置 setMessageLogging() 用来开启 debug 工作

```
public static void loop() {
    final Looper me = myLooper();          // 获取 TLS 存储的 Looper 对象 【见2.4】
    final MessageQueue queue = me.mQueue;  // 获取 Looper 对象中的消息队列

    Binder.clearCallingIdentity();
    //确保在权限检查时基于本地进程，而不是调用进程。
    final long ident = Binder.clearCallingIdentity();

    for (;;) {  // 进入loop的主循环方法
        Message msg = queue.next(); // 可能会阻塞 【见4.2】
        if (msg == null) {   // 没有消息，则退出循环
            return;
        }
        
        // 默认为 null，可通过 setMessageLogging() 方法来指定输出，用于 debug 功能
        Printer logging = me.mLogging;  
        if (logging != null) {
            logging.println(">>>>> Dispatching to " + msg.target + " " +
                    msg.callback + ": " + msg.what);
        }
        msg.target.dispatchMessage(msg); // 用于分发 Message 【见3.2】
        if (logging != null) {
            logging.println("<<<<< Finished to " + msg.target + " " + msg.callback);
        }
        
        // 恢复调用者信息
        final long newIdent = Binder.clearCallingIdentity();
        msg.recycleUnchecked();  // 将 Message 放入消息池 【见5.2】
    }
}
```

### 2.3 quit()
- Looper.quit() 方法的实现最终调用的是 MessageQueue.quit() 方法

```
public void quit() {
    mQueue.quit(false);  // 消息移除
}

public void quitSafely() {
    mQueue.quit(true);   // 安全地消息移除
}
```

- MessageQueue.quit()
  - 当 safe = true 时，只移除尚未触发的所有消息，对于正在触发的消息并不移除
  - 当 safe = flase 时，移除所有的消息

```
void quit(boolean safe) {
    // 当 mQuitAllowed 为 false，表示不运行退出，强行调用 quit() 会抛出异常
    if (!mQuitAllowed) {
        throw new IllegalStateException("Main thread not allowed to quit.");
    }
    synchronized (this) {
        if (mQuitting) { 
            // 防止多次执行退出操作
            return;
        }
        mQuitting = true;
        if (safe) {
            removeAllFutureMessagesLocked(); // 移除尚未触发的所有消息
        } else {
            removeAllMessagesLocked();       // 移除所有的消息
        }
        
        // mQuitting=false，那么认定为 mPtr != 0
        nativeWake(mPtr);
    }
}
```

### 2.4 常用方法
#### 2.4.1 myLooper
- 用于获取 TLS 存储的 Looper 对象


```
public static @Nullable Looper myLooper() {
    return sThreadLocal.get();
}
```

#### 2.4.2 post
- 发送消息，并设置消息的 callback，用于处理消息


```
public final boolean post(Runnable r) {
   return sendMessageDelayed(getPostMessage(r), 0);
}

private static Message getPostMessage(Runnable r) {
    Message m = Message.obtain();
    m.callback = r;
    return m;
}
```

---

## 3. Handler
### 3.1 创建 Handler
#### 3.1.1 无参构造
- 默认采用当前线程 TLS 中的 Looper 对象，并且 callback 回调方法为 null，且消息为同步处理方式
- 只要执行的 Looper.prepare() 方法，那么便可以获取有效的 Looper 对象


```
public Handler() {
    this(null, false);
}

public Handler(Callback callback, boolean async) {
    //匿名类、内部类或本地类都必须申明为static，否则会警告可能出现内存泄露
    if (FIND_POTENTIAL_LEAKS) {
        final Class<? extends Handler> klass = getClass();
        if ((klass.isAnonymousClass() || klass.isMemberClass() || klass.isLocalClass()) &&
                (klass.getModifiers() & Modifier.STATIC) == 0) {
            Log.w(TAG, "The following Handler class should be static or leaks might occur: " +
                klass.getCanonicalName());
        }
    }
    //必须先执行Looper.prepare()，才能获取Looper对象，否则为null.
    mLooper = Looper.myLooper();  //从当前线程的TLS中获取Looper对象【见2.1】
    if (mLooper == null) {
        throw new RuntimeException("");
    }
    mQueue = mLooper.mQueue; //消息队列，来自Looper对象
    mCallback = callback;  //回调方法
    mAsynchronous = async; //设置消息是否为异步处理方式
}
```

#### 3.1.2 有参构造
- 可指定 Looper，Callback 回调方法以及消息的处理方式(同步或异步)

```
public Handler(Looper looper) {
    this(looper, null, false);
}

public Handler(Looper looper, Callback callback, boolean async) {
    mLooper = looper;
    mQueue = looper.mQueue;
    mCallback = callback;
    mAsynchronous = async;
}
```

### 3.2 消息分发机制
- 分发消息流程：
  - 当 Message 的回调方法不为空时，则回调方法 msg.callback.run()，其中 callBack 数据类型为 Runnable, 否则进入步骤2
  - 当 Handler 的 mCallback 成员变量不为空时，则回调方法 mCallback.handleMessage(msg), 否则进入步骤3
  - 调用 Handler 自身的回调方法 handleMessage()，该方法默认为空，Handler 子类通过覆写该方法来完成具体的逻辑


```
public void dispatchMessage(Message msg) {
    if (msg.callback != null) {
        // 当 Message 存在回调方法，回调 msg.callback.run() 方法；
        handleCallback(msg);
    } else {
        if (mCallback != null) {
            // 当 Handler 存在 Callback 成员变量时，回调方法 handleMessage()；
            if (mCallback.handleMessage(msg)) {
                return;
            }
        }
        // Handler 自身的回调方法 handleMessage()
        handleMessage(msg);
    }
}
```

### 3.3 消息发送
![image](handler1_page2.png)


#### 3.3.6 post
```
// 结合 dispatchMessage 方法，可知 callback 优先级最高
public final boolean post(Runnable r) {
   return  sendMessageDelayed(getPostMessage(r), 0);
}

private static Message getPostMessage(Runnable r) {
    Message m = Message.obtain();
    m.callback = r;
    return m;
}
```

#### 3.3.8 enqueueMessage
```
private boolean enqueueMessage(MessageQueue queue, Message msg, long uptimeMillis) {
    msg.target = this;
    if (mAsynchronous) {
        msg.setAsynchronous(true);
    }
    return queue.enqueueMessage(msg, uptimeMillis); 【见4.3】
}
```

### 3.4 Handler其他方法
#### 3.4.1 obtainMessage
```
public final Message obtainMessage() {
    return Message.obtain(this); 【见5.2】
}
```

#### 3.4.2 removeMessages
```
public final void removeMessages(int what) {
    mQueue.removeMessages(this, what, null); 【见 4.5】
}
```

---

## 4. MessageQueue
- MessageQueue 是消息机制的 Java 层和 C++ 层的连接纽带，大部分核心方法都交给 native 层来处理


```
private native static long nativeInit();
private native static void nativeDestroy(long ptr);
private native void nativePollOnce(long ptr, int timeoutMillis);
private native static void nativeWake(long ptr);
private native static boolean nativeIsPolling(long ptr);
private native static void nativeSetFileDescriptorEvents(long ptr, int fd, int events);
```

### 4.1 创建MessageQueue
```
MessageQueue(boolean quitAllowed) {
    mQuitAllowed = quitAllowed;
    //通过native方法初始化消息队列，其中mPtr是供native代码使用
    mPtr = nativeInit();
}
```

### 4.2 next()
- nativePollOnce 是阻塞操作，其中 nextPollTimeoutMillis 代表下一个消息到来前，还需要等待的时长；当 nextPollTimeoutMillis = -1时，表示消息队列中无消息，会一直等待下去
- 当处于空闲时，往往会执行 IdleHandler 中的方法。当 nativePollOnce() 返回后，next() 从 mMessages 中提取一个消息

```
Message next() {
    final long ptr = mPtr;
    if (ptr == 0) { 
        // 当消息循环已经退出，则直接返回
        return null;
    }
    int pendingIdleHandlerCount = -1; // 循环迭代的首次为-1
    int nextPollTimeoutMillis = 0;
    for (;;) {
        if (nextPollTimeoutMillis != 0) {
            Binder.flushPendingCommands();
        }
        // 阻塞操作，当等待 nextPollTimeoutMillis 时长，或者消息队列被唤醒，都会返回
        nativePollOnce(ptr, nextPollTimeoutMillis);
        synchronized (this) {
            final long now = SystemClock.uptimeMillis();
            Message prevMsg = null;
            Message msg = mMessages;
            if (msg != null && msg.target == null) {
                // 当消息 Handler 为空时，查询 MessageQueue 中的下一条异步消息 msg，则退出循环。
                do {
                    prevMsg = msg;
                    msg = msg.next;
                } while (msg != null && !msg.isAsynchronous());
            }
            if (msg != null) {
                if (now < msg.when) {
                    // 当异步消息触发时间大于当前时间，则设置下一次轮询的超时时长
                    nextPollTimeoutMillis = (int) Math.min(msg.when - now, Integer.MAX_VALUE);
                } else {
                    // 获取一条消息，并返回
                    mBlocked = false;
                    if (prevMsg != null) {
                        prevMsg.next = msg.next;
                    } else {
                        mMessages = msg.next;
                    }
                    msg.next = null;
                    // 设置消息的使用状态，即flags |= FLAG_IN_USE
                    msg.markInUse();
                    return msg;   // 成功地获取 MessageQueue 中的下一条即将要执行的消息
                }
            } else {
                // 没有消息
                nextPollTimeoutMillis = -1;
            }
            // 消息正在退出，返回 null
            if (mQuitting) {
                dispose();
                return null;
            }
            // 当消息队列为空，或者是消息队列的第一个消息时
            if (pendingIdleHandlerCount < 0 && (mMessages == null || now < mMessages.when)) {
                pendingIdleHandlerCount = mIdleHandlers.size();
            }
            if (pendingIdleHandlerCount <= 0) {
                // 没有 idle handlers 需要运行，则循环并等待。
                mBlocked = true;
                continue;
            }
            if (mPendingIdleHandlers == null) {
                mPendingIdleHandlers = new IdleHandler[Math.max(pendingIdleHandlerCount, 4)];
            }
            mPendingIdleHandlers = mIdleHandlers.toArray(mPendingIdleHandlers);
        }
        
        // 只有第一次循环时，会运行 idle handlers，执行完成后，重置 pendingIdleHandlerCount 为 0.
        for (int i = 0; i < pendingIdleHandlerCount; i++) {
            final IdleHandler idler = mPendingIdleHandlers[i];
            mPendingIdleHandlers[i] = null; // 去掉 handler 的引用
            boolean keep = false;
            try {
                keep = idler.queueIdle();   // idle 时执行的方法
            } catch (Throwable t) {
                Log.wtf(TAG, "IdleHandler threw exception", t);
            }
            if (!keep) {
                synchronized (this) {
                    mIdleHandlers.remove(idler);
                }
            }
        }
        
        // 重置 idle handler 个数为 0，以保证不会再次重复运行
        pendingIdleHandlerCount = 0;
        
        // 当调用一个空闲 handler 时，一个新 message 能够被分发，因此无需等待可以直接查询 pending message.
        nextPollTimeoutMillis = 0;
    }
}
```

### 4.3 enqueueMessage
```
boolean enqueueMessage(Message msg, long when) {
    // 每一个普通 Message 必须有一个 target
    if (msg.target == null) {
        throw new IllegalArgumentException("Message must have a target.");
    }
    if (msg.isInUse()) {
        throw new IllegalStateException(msg + " This message is already in use.");
    }
    synchronized (this) {
        if (mQuitting) {
            // 正在退出时，回收 msg，加入到消息池
            msg.recycle();
            return false;
        }
        msg.markInUse();
        msg.when = when;
        Message p = mMessages;
        boolean needWake;
        if (p == null || when == 0 || when < p.when) {
            // p 为 null(代表 MessageQueue 没有消息）或者 msg 的触发时间是队列中最早的， 则进入该该分支
            msg.next = p;
            mMessages = msg;
            needWake = mBlocked; // 当阻塞时需要唤醒
        } else {
            // 将消息按时间顺序插入到 MessageQueue。一般地，不需要唤醒事件队列，除非
            // 消息队头存在 barrier，并且同时 Message 是队列中最早的异步消息。
            needWake = mBlocked && p.target == null && msg.isAsynchronous();
            Message prev;
            for (;;) {
                prev = p;
                p = p.next;
                if (p == null || when < p.when) {
                    break;
                }
                if (needWake && p.isAsynchronous()) {
                    needWake = false;
                }
            }
            msg.next = p;
            prev.next = msg;
        }
        
        // 消息没有退出，我们认为此时 mPtr != 0
        if (needWake) {
            nativeWake(mPtr);
        }
    }
    return true;
}
```

---

## 5. Message
### 5.1 消息对象
![image](handler1_page3.png)

### 5.2 消息池
- 当消息池不为空时，可以直接从消息池中获取 Message 对象，而不是直接创建，提高效率
- 静态变量 sPool 的数据类型为 Message，通过 next 成员变量，维护一个消息池；
- 静态变量 MAX_POOL_SIZE 代表消息池的可用大小；
- 消息池的默认大小为 50

#### 5.2.1 obtain
```
public static Message obtain() {
    synchronized (sPoolSync) {
        if (sPool != null) {
            Message m = sPool;
            sPool = m.next;
            m.next = null;   // 从 sPool 中取出一个 Message 对象，并消息链表断开
            m.flags = 0;     // 清除 in-use flag
            sPoolSize--;     // 消息池的可用大小进行减1操作
            return m;
        }
    }
    return new Message();    // 当消息池为空时，直接创建 Message 对象
}
```

---

## 总结
### 图解
- Handler 通过 sendMessage() 发送 Message 到 MessageQueue 队列
- Looper 通过 loop()，不断提取出达到触发条件的 Message，并将 Message 交给 target 来处理
- 经过 dispatchMessage() 后，交回给 Handler 的 handleMessage() 来进行相应地处理
- 将 Message 加入 MessageQueue 时，会往管道写入字符，可以会唤醒 loop 线程；如果 MessageQueue 中没有 Message，并处于 Idle 状态，则会执行 IdelHandler 接口中的方法，往往用于做一些清理性地工作

![image](handler1_page4.png)

### 消息分发的优先级
1. Message 的回调方法：message.callback.run()，优先级最高
2. Handler 的回调方法：Handler.mCallback.handleMessage(msg)，优先级仅次于1
3. Handler 的默认方法：Handler.handleMessage(msg)，优先级最低

### 消息缓存
- 为了提供效率，提供了一个大小为 50 的 Message 缓存队列，减少对象不断创建与销毁的过程

---

## 补充
### 小结
1. Looper 依赖于 MessageQueue 和 Thread，每个 Thread 只对应一个 Looper，每个 Looper 只对应一个 MessageQueue（一对一）
2. MessageQueue 依赖于 Message，每个 MessageQueue 中有 N 个待处理消息（一对N）
3. Message 依赖于 Handler 来进行处理，每个 Message 有且仅有一个对应的 Handler。（一对一）
4. Handler 中持有 Looper 和 MessageQueue 的引用，可直接对其进行操作

![image](handler1_page5.png)
![image](handler1_page6.png)

---

## 补充：Java 层、native 层
- Java 层和 native 层通过 MessageQueue 里面持有一个 native 的 MessageQueue 对象进行交互。WeakMessageHandler 继承自MessageHandler，NativeMessageQueue 继承自 MessageQueue
- Java 层和 native 层实质是各自维护了一套相似的消息系统。C 层发出的消息和 Java 层发出的消息可以没有任何关系。所以 Framework 层只是很巧的利用了底层 epoll 的机制达到阻塞的目的
- 通过 pollOnce 的分析，可以发现消息的处理其实是有顺序的，首先是处理 native message，然后处理 native request，最后才会执行 java 层，处理 java 层的 message

![image](handler1_page7.png)


## 参考文档
[博客](http://gityuan.com/2015/12/26/handler-message-framework/)
[简书](https://www.jianshu.com/p/8656bebc27cb)

---